In [13]:
import numpy as np
import cv2
import os

In [14]:
def generate_frames(video, face_id):
    cam = cv2.VideoCapture(video)
    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    count = 0
    while True:
        ret, img = cam.read()
        if not ret:
            print("Failed to capture video.")
            break

        if face_detector.empty():
            print("Error: Unable to load the Haar Cascade XML file.")
            exit()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
            count += 1
            cv2.imwrite("dataset/User." + str(face_id) + '.' +
                        str(count) + ".jpg", gray[y:y+h, x:x+w])
        if count >= 30:  
            break

In [15]:
def generate_dataset():
    cnt = 1
    for video in os.listdir('videos'):
        path  = os.path.join('videos', video)
        generate_frames(path, cnt)
        cnt+=1

In [16]:
# generate_dataset()